In [1]:
%run 01_setup.ipynb

23/11/16 12:47:12 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [14]:
%%sparksql
-- take only clear data
CREATE OR REPLACE TEMP VIEW ex_clear AS
    SELECT * FROM ex WHERE date(srch_ci) <= date(srch_co);

In [15]:
%%sparksql
-- sum booked rooms by dates
CREATE OR REPLACE TEMP VIEW visits AS
    WITH v AS (
        SELECT 
            hotel_id,
            explode(sequence(date(srch_ci),date(srch_co), INTERVAL 1 DAY)) AS date,
            srch_rm_cnt
        FROM ex_clear)
    SELECT hotel_id, date, sum(srch_rm_cnt) AS used_rooms FROM v GROUP BY hotel_id, date;

In [26]:
%%sparksql
WITH ymCTE(year, month) AS (
    SELECT DISTINCT year(date) AS year, month(date) AS month 
    FROM visits
)
SELECT hotel_id, ymCTE.year, ymCTE.month FROM visits LIMIT 10

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `ymCTE`.`year` cannot be resolved. Did you mean one of the following? [`visits`.`date`, `visits`.`hotel_id`, `visits`.`used_rooms`].; line 5 pos 17;
'WithCTE
:- CTERelationDef 18, false
:  +- SubqueryAlias ymCTE
:     +- Project [year#326 AS year#402, month#327 AS month#403]
:        +- Distinct
:           +- Project [year(date#331) AS year#326, month(date#331) AS month#327]
:              +- SubqueryAlias visits
:                 +- View (`visits`, [hotel_id#330L,date#331,used_rooms#332L])
:                    +- Project [cast(hotel_id#352L as bigint) AS hotel_id#330L, cast(date#353 as date) AS date#331, cast(used_rooms#328L as bigint) AS used_rooms#332L]
:                       +- WithCTE
:                          :- CTERelationDef 19, false
:                          :  +- SubqueryAlias v
:                          :     +- Project [hotel_id#352L, date#353, srch_rm_cnt#349]
:           

In [50]:
spark.sql("SELECT * FROM visits").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[hotel_id#47L, date#613], functions=[sum(srch_rm_cnt#44)])
   +- HashAggregate(keys=[hotel_id#47L, date#613], functions=[partial_sum(srch_rm_cnt#44)])
      +- Project [hotel_id#47L, date#613, srch_rm_cnt#44]
         +- Generate explode(sequence(cast(srch_ci#40 as date), cast(srch_co#41 as date), Some(INTERVAL '1' DAY), Some(Europe/Kyiv))), [srch_rm_cnt#44, hotel_id#47L], false, [date#613]
            +- Filter ((isnotnull(srch_ci#40) AND isnotnull(srch_co#41)) AND (cast(srch_ci#40 as date) <= cast(srch_co#41 as date)))
               +- GlobalLimit 1000, 0
                  +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=1268]
                     +- LocalLimit 1000
                        +- FileScan avro [srch_ci#40,srch_co#41,srch_rm_cnt#44,hotel_id#47L] Batched: false, DataFilters: [], Format: Avro, Location: InMemoryFileIndex(1 paths)[abfss://data@styakovdwesteurope.dfs.core.windows.net/expe

In [ ]:
%%sparksql
CREATE OR REPLACE TEMPORARY VIEW hwp AS
    SELECT /*+ BROADCAST(periods) */ * FROM periods JOIN hw ON TRUE;